# Setup Environment

In [2]:
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path_to_add = os.path.join(parent_dir, "src")
sys.path.insert(0, path_to_add)

# Get Data

In [3]:
import requests
import pandas as pd
import janitor

# URL of the endpoint
url = "https://data.austintexas.gov/resource/jeyv-db9u.json"

# Fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(response.json()).clean_names()
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [4]:
%load_ext autoreload

In [68]:
%autoreload 2
from pandas_survey_toolkit import analytics, nlp, vis
from pandas_survey_toolkit.vis import create_keyword_graph, visualize_keyword_graph,visualize_keyword_graph_force, create_keyword_sentiment_df_simple, create_sentiment_color_mapping

In [70]:
df2 = df.extract_keywords("response", top_n=4, ngram_range=(1,1), min_df=4).extract_sentiment("response")

In [71]:
df2.keywords

0                     [level, manage]
1                     [like, theater]
2                                  []
3      [need, expansion, accommodate]
4                            [update]
                    ...              
857        [staff, helpful, friendly]
858                                []
859                            [hope]
860      [feature, artist, welcoming]
861            [amazing, opportunity]
Name: keywords, Length: 862, dtype: object

In [72]:
sentiment_df = create_keyword_sentiment_df_simple(df2)
sentiment_df = sentiment_df.query("word != 'not'")
color_mapping = create_sentiment_color_mapping(sentiment_df)

G = create_keyword_graph(df2, 'keywords', node_color_mapping=color_mapping)
G.remove_node("not")
visualize_keyword_graph_force(G, output_file='keyword_sentiment_graph.html', colormap='RdBu', min_edge_count=2, min_node_count=1)

Graph saved to keyword_sentiment_graph.html
